In [1]:
import numpy as np
import pandas as pd

pd.set_option('display.precision', 15)  # Increase decimal precision
pd.set_option('display.width', 150)     # Wider display
pd.set_option('display.max_columns', None)  # Show all column

# Phương pháp tiếp tuyến (Newton-Raphson)

Điều kiện:

* $(a,b)$ là khoảng cách ly nghiệm

* $f'(x)$ xác định dấu không đổi trên $[a,b]$

* $f''(x)$ xác định dấu không đổi trên $[a,b]$

Điều kiện hội tụ:

* Chọn đúng điểm xuất phát ban đầu $x_0$ (điểm Fourier - $f(x_0) \cdot f''(x_0) > 0$)

# Thuật toán:

1. Chọn đúng điểm xuất phát ban đầu $x_0$ (điểm Fourier - $f(x_0) \cdot f''(x_0) > 0$)

2. Xác định $x$ theo công thức lặp: $x_{k+1} = x_k - \dfrac{f(x_k)}{f'(x_k)}$

3. Lặp lại bước (2) cho đến khi:

   a. $x_n$ là nghiệm của phương trình, hoặc

   b. dãy $\{x_n\}$ thỏa mãn điều kiện dừng

5. In giá trị $x_n$ (giá trị $x$ sau $n$ lần lặp)

# Áp dụng

## 1. Hậu nghiêm - Số lần lặp

Kiểm tra sai số theo 1 trong 2 công thức sai số sau:
+ $|x_n - x^*| \leqslant \dfrac{|f(x_n)|}{m_1}$ $\quad\quad$ (1)

+ $|x_n - x^*| \leqslant \dfrac{M_2}{2 m_1} \cdot |x_n - x_{n-1}|^2$ $\quad\quad$ (2)

với $m_1 = \min\limits_{[a,b]} |f'(x)|, \quad M_2 = \max\limits_{[a,b]} |f''(x)|$

### 1.1. Công thức 1

In [2]:
def newton_iteration_v1 (f, df, d2f, a, b, n, rbl):	
	M2 = max([np.abs(d2f(x)) for x in [a, b]]) #M2 is the maximum value of |f''(x)| in the interval [a,b]
	m1 = min([np.abs(df(x)) for x in [a, b]]) #m1 is the minimum value of |f'(x)| in the interval [a,b]
	print (f"m1 = {m1}, M2 = {M2}")

	#Starting values
	if f(a) * d2f(a) > 0:
		x = a
	elif f(b) * d2f(b) > 0:
		x = b
	
	results = []
	results.append({
        'n': 0,
        'x_n': x,
        'f(x_n)': f(x),
        'delta_x=|f(x_n)| / m_1': abs(f(x)) / m1
    })

	#Newton's method
	x_new = 0; delta_x = 0;
	for i in range(n):
		# calculate the next value of x
		x_new = x - f(x) / df(x)
		delta_x = abs(f(x_new)) / m1

		results.append({
            'n': i+1,
            'x_n': x_new,
            'f(x_n)': f(x_new),
            'delta_x=|f(x_n)| / m_1': delta_x
        })

		# Prepare for next iteration
		x = x_new
		if (f(x_new) == 0): 
			break

	#Print the final result
	df_results = pd.DataFrame(results)
	print(df_results.to_string(index=False))

	if rbl == None:
		print(f"The value of root is: {x}")
	else:
		total_delta = delta_x + 0.5 * 10**(-rbl)
		print(f"The value of root with {rbl} decimal point is: {round(x, rbl)}")
		print(f"Relative error is: {total_delta}")

In [3]:
f = lambda x: x**4-27
df = lambda x: 4*x**3
d2f = lambda x: 12*x**2

a = 2
b = 3

n = 5
rbl = 9

newton_iteration_v1 (f, df, d2f, a, b, n, rbl);

m1 = 32, M2 = 108
 n               x_n             f(x_n)  delta_x=|f(x_n)| / m_1
 0 3.000000000000000 54.000000000000000       1.687500000000000
 1 2.500000000000000 12.062500000000000       0.376953125000000
 2 2.307000000000000  1.326334418000997       0.041447950562531
 3 2.279994621743364  0.023107580939026       0.000722111904345
 4 2.279507213327209  0.000007408717718       0.000000231522429
 5 2.279507056954794  0.000000000000771       0.000000000000024
The value of root with 9 decimal point is: 2.279507057
Relative error is: 5.000240918396344e-10


### 1.2. Công thức 2

In [4]:
def newton_iteration_v2 (f, df, d2f, a, b, n, rbl):	
	M2 = max([np.abs(d2f(x)) for x in [a, b]]) #M2 is the maximum value of |f''(x)| in the interval [a,b]
	m1 = min([np.abs(df(x)) for x in [a, b]]) #m1 is the minimum value of |f'(x)| in the interval [a,b]
	print (f"m1 = {m1}, M2 = {M2}")

	#Starting values
	if f(a) * d2f(a) > 0:
		x = a
	elif f(b) * d2f(b) > 0:
		x = b
	
	results = []
	results.append({
        'n': 0,
        'x_n': x,
        'f(x_n)': f(x),
        'delta_x=M_2/(2*m_1) * |x_n - x_(n-1)|^2': None
    })

	#Newton's method
	x_new = 0; delta_x = 0;
	for i in range(n):
		# calculate the next value of x
		x_new = x - f(x) / df(x)
		delta_x = M2 / (2 * m1) * (x_new - x)**2

		results.append({
            'n': i+1,
            'x_n': x_new,
            'f(x_n)': f(x_new),
            'delta=M_2/(2*m_1) * |x_n - x_(n-1)|^2': delta_x
        })

		# Prepare for next iteration
		x = x_new
		if (f(x_new) == 0): 
			break

	#Print the final result
	df_results = pd.DataFrame(results)
	print(df_results.to_string(index=False))

	if rbl == None:
		print(f"The value of root is: {x}")
	else:
		total_delta = delta_x + 0.5 * 10**(-rbl)
		print(f"The value of root with {rbl} decimal point is: {round(x, rbl)}")
		print(f"Relative error is: {total_delta}")

In [5]:
f = lambda x: x**4-27
df = lambda x: 4*x**3
d2f = lambda x: 12*x**2

a = 2
b = 3

n = 5
rbl = 9

newton_iteration_v2 (f, df, d2f, a, b, n, rbl);

m1 = 32, M2 = 108
 n               x_n             f(x_n)  delta_x=M_2/(2*m_1) * |x_n - x_(n-1)|^2  delta=M_2/(2*m_1) * |x_n - x_(n-1)|^2
 0 3.000000000000000 54.000000000000000                                      NaN                                    NaN
 1 2.500000000000000 12.062500000000000                                      NaN                      0.421875000000000
 2 2.307000000000000  1.326334418000997                                      NaN                      0.062857687500000
 3 2.279994621743364  0.023107580939026                                      NaN                      0.001230677642448
 4 2.279507213327209  0.000007408717718                                      NaN                      0.000000400894252
 5 2.279507056954794  0.000000000000771                                      NaN                      0.000000000000041
The value of root with 9 decimal point is: 2.279507057
Relative error is: 5.000412633105506e-10


## 2. Hậu nghiệm - Sai số tương đối

Thiết lập điều kiện dừng theo 1 trong 2 công thức sai số sau:
+ $|x_n - x^*| \leqslant \dfrac{|f(x_n)|}{m_1} < \epsilon$ $\quad\quad (1)$

+ $|x_n - x^*| \leqslant \dfrac{M_2}{2 m_1} \cdot |x_n - x_{n-1}|^2 < \epsilon$ $\quad\quad (2)$

với $m_1 = \min\limits_{[a,b]} |f'(x)|, \quad M_2 = \max\limits_{[a,b]} |f''(x)|$


### 2.1. Công thức 1

Từ công thức (1) suy ra Điều kiện dừng: $f(x_n)| < m_1 \epsilon$

In [6]:
def newton_recursion_absolute_v1 (f, df, d2f, a, b, eps):	
	M2 = max([np.abs(d2f(x)) for x in [a, b]]) #M2 is the maximum value of |f''(x)| in the interval [a,b]
	m1 = min([np.abs(df(x)) for x in [a, b]]) #m1 is the minimum value of |f'(x)| in the interval [a,b]
	print (f"m1 = {m1}, M2 = {M2}")

	#Starting values
	if f(a) * d2f(a) > 0:
		x = a
	elif f(b) * d2f(b) > 0:
		x = b
	
	results = []
	results.append({
        'n': 0,
        'x_n': x,
        'f(x_n)': f(x),
		"delta_x=|f(x_n)|": abs(f(x))
    })

	#Newton's method
	x_new = 0; new_eps = m1*eps;
	print(f"delta_x = {new_eps}")

	i=0;
	while True:
		# calculate the next value of x
		x_new = x - f(x) / df(x)
		current_delta_x = abs(f(x_new))

		results.append({
            'n': i+1,
            'x_n': x_new,
            'f(x_n)': f(x_new),
            'delta_x=|f(x_n)|': current_delta_x
        })
		
		# update the value of interval 
		x = x_new
		if (f(x_new) == 0): 
			break

		#stop condition
		if current_delta_x < new_eps:
			break
		else:
			i += 1

	#Print the final result
	df_results = pd.DataFrame(results)
	print(df_results.to_string(index=False))


	print(f"The value of root with absolute error {eps} is: {x}")

In [7]:
f = lambda x: x**5 - np.log(x) - 12
df = lambda x: 5*x**4 - 1/x
d2f = lambda x: 20*x**3 + 1/x**2

a = 1
b = 2

eps = 10**(-8)

newton_recursion_absolute_v1 (f, df, d2f, a, b, eps);

m1 = 4.0, M2 = 160.25
delta_x = 4e-08
 n               x_n             f(x_n)   delta_x=|f(x_n)|
 0 2.000000000000000 19.306852819440056 19.306852819440056
 1 1.757146505415848  4.187275139066788  4.187275139066788
 2 1.668237426251162  0.409029295346730  0.409029295346730
 3 1.657509199354213  0.005330058958570  0.005330058958570
 4 1.657365671196235  0.000000941752862  0.000000941752862
 5 1.657365645827691  0.000000000000027  0.000000000000027
The value of root with absolute error 1e-08 is: 1.6573656458276909


### 2.2. Công thức 2

Từ công thức (2) suy ra Điều kiện dừng: $|x_n - x_{n-1}| < \sqrt{2 \epsilon \cdot \dfrac{m_1}{M_2}}$

In [8]:
def newton_recursion_absolute_v2 (f, df, d2f, a, b, eps):	
	M2 = max([np.abs(d2f(x)) for x in [a, b]]) #M2 is the maximum value of |f''(x)| in the interval [a,b]
	m1 = min([np.abs(df(x)) for x in [a, b]]) #m1 is the minimum value of |f'(x)| in the interval [a,b]
	print (f"m1 = {m1}, M2 = {M2}")

	#Starting values
	if f(a) * d2f(a) > 0:
		x = a
	elif f(b) * d2f(b) > 0:
		x = b
	
	results = []
	results.append({
        'n': 0,
        'x_n': x,
        'f(x_n)': f(x),
        'delta_x=|x_n - x_(n-1)|': None
    })

	#Newton's method
	x_new = 0; new_eps = np.sqrt(eps * 2 * m1 / M2);
	print(f"delta_x = {new_eps}")

	i=0;
	while True:
		# calculate the next value of x
		x_new = x - f(x) / df(x)
		current_delta_x = abs(x-x_new)

		results.append({
            'n': i+1,
            'x_n': x_new,
            'f(x_n)': f(x_new),
            'delta_x=|x_n - x_(n-1)|': current_delta_x
        })
		
		# update the value of interval 
		x = x_new
		if (f(x_new) == 0): 
			break

		#stop condition
		if current_delta_x < new_eps:
			break
		else:
			i += 1

	#Print the final result
	df_results = pd.DataFrame(results)
	print(df_results.to_string(index=False))


	print(f"The value of root with absolute error {eps} is: {x}")

In [9]:
f = lambda x: x**4-27
df = lambda x: 4*x**3
d2f = lambda x: 12*x**2

a = 2
b = 3

eps = 0.5 * 10**(-7)

newton_recursion_absolute_v2 (f, df, d2f, a, b, eps);

m1 = 32, M2 = 108
delta_x = 0.00017213259316477408
 n               x_n             f(x_n)  delta_x=|x_n - x_(n-1)|
 0 3.000000000000000 54.000000000000000                      NaN
 1 2.500000000000000 12.062500000000000        0.500000000000000
 2 2.307000000000000  1.326334418000997        0.193000000000000
 3 2.279994621743364  0.023107580939026        0.027005378256636
 4 2.279507213327209  0.000007408717718        0.000487408416155
 5 2.279507056954794  0.000000000000771        0.000000156372415
The value of root with absolute error 5e-08 is: 2.279507056954794


## 3. Hậu nghiệm - Sai số tuyệt đối

Thiết lập điều kiện dừng theo công thức sai số: $\dfrac{|x_n - x^*|}{|x_n|} \leqslant \dfrac{M_2}{2 m_1} \cdot \dfrac{|x_n - x_{n-1}|^2}{|x_n|} < \eta$

với $m_1 = \min\limits_{[a,b]} |f'(x)|, \quad M_2 = \max\limits_{[a,b]} |f''(x)|$

### 3.1. Công thức mục tiêu
Từ công thức sai số hậu nghiệm, ta có Điều kiện dừng: $\dfrac{|x_n - x_{n-1}|^2}{|x_n|} < 2 \eta \cdot \dfrac{m_1}{M_2}$

In [10]:
def newton_recursion_relative (f, df, d2f, a, b, eta):	
	M2 = max([np.abs(d2f(x)) for x in [a, b]]) #M2 is the maximum value of |f''(x)| in the interval [a,b]
	m1 = min([np.abs(df(x)) for x in [a, b]]) #m1 is the minimum value of |f'(x)| in the interval [a,b]
	print (f"m1 = {m1}, M2 = {M2}")

	#Starting values
	if f(a) * d2f(a) > 0:
		x = a
	elif f(b) * d2f(b) > 0:
		x = b
	
	results = []
	results.append({
        'n': 0,
        'x_n': x,
        'f(x_n)': f(x),
        'sigma_x=|x_n-x_(n-1)|^2/|x_n|': None
    })

	#Newton's method
	x_new = 0; new_eta = 2 * eta * m1 / M2;
	print(f"sigma_x = {new_eta}")

	i=0;
	while True:
		# calculate the next value of x
		x_new = x - f(x) / df(x)
		current_sigma_x = (x-x_new)**2 / abs(x_new)

		results.append({
            'n': i+1,
            'x_n': x_new,
            'f(x_n)': f(x_new),
            'delta_x=|x_n - x_(n-1)|': current_sigma_x
        })
		
		# update the value of interval 
		x = x_new
		if (f(x_new) == 0): 
			break

		#stop condition
		if current_sigma_x < new_eta:
			break
		else:
			i += 1

	#Print the final result
	df_results = pd.DataFrame(results)
	print(df_results.to_string(index=False))


	print(f"The value of root with relative error {eta} is: {x}")

In [11]:
f = lambda x: x**4-27
df = lambda x: 4*x**3
d2f = lambda x: 12*x**2

a = 2
b = 3

eta = 0.5 * 10**(-7)

newton_recursion_relative (f, df, d2f, a, b, eta);

m1 = 32, M2 = 108
sigma_x = 2.962962962962963e-08
 n               x_n             f(x_n)  sigma_x=|x_n-x_(n-1)|^2/|x_n|  delta_x=|x_n - x_(n-1)|
 0 3.000000000000000 54.000000000000000                            NaN                      NaN
 1 2.500000000000000 12.062500000000000                            NaN        0.100000000000000
 2 2.307000000000000  1.326334418000997                            NaN        0.016146077156480
 3 2.279994621743364  0.023107580939026                            NaN        0.000319864989079
 4 2.279507213327209  0.000007408717718                            NaN        0.000000104218562
 5 2.279507056954794  0.000000000000771                            NaN        0.000000000000011
The value of root with relative error 5e-08 is: 2.279507056954794
